In [112]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-06-28 12:39:46,575: Credentials are already in use. The existing account in the session will be replaced.


In [113]:
from qiskit.circuit.library.arithmetic.adders import CDKMRippleCarryAdder

In [114]:
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

In [115]:
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-06-28 12:39:51,130: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [116]:
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

In [117]:
provider = IBMQ.get_provider('ibm-q')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_oslo

In [118]:
import qiskit.tools.jupyter

backend_ex = provider.get_backend('ibm_oslo')
backend_ex

<IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>

In [119]:
backends = provider.backends(filters = lambda x:x.configuration().n_qubits >= 2 and not x.configuration().simulator and x.status().operational==True)
backends


[<IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [120]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 and not x.configuration().simulator and x.status().operational==True))
backend

<IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>

In [121]:
# run this cell
backend = provider.get_backend('ibm_oslo')

In [122]:
from qiskit.tools.monitor import job_monitor

In [123]:
 adder = CDKMRippleCarryAdder(2,kind='full').to_gate()

In [131]:
def RCA3(a0, a1, b0, b1, backend): 
    
    qca = QuantumCircuit(6,3)
    qca.reset(range(3))
    backend = provider.get_backend(backend)
    
    # input 
    if a0=='1':
        qca.x(0)
    if a1=='1':
        qca.x(1)
    if b0=='1':
        qca.x(2)
    if b1=='1':
        qca.x(3)
        
    qca.barrier()
    
    #operation
    qca.append(adder,[0,1,2,3,4,5])
    
    qca.barrier()
    
    qca.measure(3,0)
    qca.measure(4,1)
    qca.measure(5,2)
    
    qca_trans = transpile(qca, backend = backend, optimization_level=3)
    job = backend.run(qca_trans, shots=8192)
    print(job.job_id())
    job_monitor(job)
    
    output = job.result().get_counts()
    
    return qca_trans, output

In [ ]:
RCA3('0','0','0','0','ibm_oslo')

62baf90cb3f348f541f253d8
Job Status: job is queued (94)    

In [ ]:
qca.draw()